In [1]:
# unit test
import unittest

import glob

import torch
from torch.utils.data import DataLoader

import segmentation_models_pytorch as smp


import sys
sys.path.insert(0, r"helper")
sys.path.insert(0, r"/helper")
sys.path.insert(0, r"./helper")
sys.path.insert(0, r"../helper")
print(sys.path)

# relevant: import the UncertaintyMetric
from compute.metric.uncertainty import * 

from dataset.meanteacher import *


['../helper', './helper', '/helper', 'helper', 'C:\\Users\\Prinzessin\\projects\\decentnet\\datasceyence\\examples', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\python39.zip', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\DLLs', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta', '', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\win32', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\Pythonwin', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Prinzessin\\.ipython']


In [2]:
class UnitTestUncertainty(unittest.TestCase):   

        
    def test_metric(self):
        
        # preparation
        channels = 1
        n_output_neurons=2
        model = smp.UnetPlusPlus(
                                encoder_name="efficientnet-b7",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
                                encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
                                in_channels=channels,          # model input channels (1 for gray-scale images, 3 for RGB, etc.)
                                classes=n_output_neurons,         # model output channels (number of classes in your dataset)
                            )
        csv_filenames = glob.glob(r"../data_prep/mt_*.csv")
        # image size has to be dividable by 32!!
        dataset = MeanTeacherTrainDataset(mode="train", channels=channels, image_size=512, csv_filenames=csv_filenames)
        train_loader = DataLoader(dataset, batch_size=3, shuffle=False)

        # relevant: create instance of class
        um = UncertaintyMetric(n_noise=4, n_repeat=2, n_output_neurons=n_output_neurons)
        uncertainy_mask = None

        # load one batch
        for idx, item in enumerate(train_loader):

            # relevant: get mask
            uncertainy_mask = um.run(model, item["img"])
            print(uncertainy_mask)

            plt.figure()
            plt.imshow( uncertainy_mask[0].permute(1, 2, 0), cmap="gray" )

            plt.figure()
            plt.imshow( uncertainy_mask[0].permute(1, 2, 0) < 0.68, cmap="gray" )

            break

    
    
    def test_dummy(self):
        
        # same code as in metric class, but with prints
        
        import torch

        #for image in batch:

        batch_size = 6
        width = 3
        height = 3
        n_copy = 2
        n_noise = 4
        output_neurons = 5
        input_channels = 1

        tmp_img = torch.rand(batch_size, input_channels, width, height) # 1 image, 2 probability outputs, 3x3 wxh
        print("action: dummy batch")
        print("batch shape", tmp_img.shape, "### batch size, input_channels, w, h")
        print()

        tmp_img_double = tmp_img.repeat(n_copy, 1, 1 ,1) # becomes 6, 2, 3, 3
        print("action: repeat dummy batch")
        print("batch shape double", tmp_img_double.shape, "### batch size * 2, input_channels, w, h")
        print()

        batch_times_repeat = tmp_img_double.shape[0] # = 6

        preds = torch.zeros( (batch_size*n_noise*n_copy), output_neurons, width, height )
        print("action: tensor for predictions")
        print("shape", preds.shape, "### n_noise * batch_size * 2 , output_neurons, w, h")
        print()

        for i in range (n_noise):

            noise = torch.clamp(torch.randn_like(tmp_img_double) * 0.1, min=-0.2, max=0.2)

            ema_input = tmp_img_double + noise

            with torch.no_grad():
                #print(n_copy * batch_size * i)
                #print(n_copy * batch_size * (i+1))
                preds[n_copy * batch_size * i    :    n_copy * batch_size * (i+1)] = i # model(input)

        preds = torch.nn.functional.softmax(preds, dim=1)


        preds = preds.reshape(n_copy*n_noise, batch_size, output_neurons, width, height)
        print("action: reshape")
        print("shape", preds.shape, " ### noise*2, batch size, output neuronns, w, h")
        print()

        preds = torch.mean(preds, dim=0)
        print("action: mean on dimension 0")
        print("shape", preds.shape, " ### batch size, output neuronns, w, h")
        print()

        uncertainty = -torch.sum(preds * torch.log(preds), dim=1, keepdim=True)
        print("action: uncertainty on dimension 1")
        print("uncertainty shape", uncertainty.shape, " ### batch size, 1, w, h")
        print()




In [3]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.

action: dummy batch
batch shape torch.Size([6, 1, 3, 3]) ### batch size, input_channels, w, h

action: repeat dummy batch
batch shape double torch.Size([12, 1, 3, 3]) ### batch size * 2, input_channels, w, h

action: tensor for predictions
shape torch.Size([48, 5, 3, 3]) ### n_noise * batch_size * 2 , output_neurons, w, h

action: reshape
shape torch.Size([8, 6, 5, 3, 3])  ### noise*2, batch size, output neuronns, w, h

action: mean on dimension 0
shape torch.Size([6, 5, 3, 3])  ### batch size, output neuronns, w, h

action: uncertainty on dimension 1
uncertainty shape torch.Size([6, 1, 3, 3])  ### batch size, 1, w, h



E
ERROR: test_metric (__main__.UnitTestUncertainty)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\Prinzessin\AppData\Local\Temp\ipykernel_18664\1160094553.py", line 31, in test_metric
    plt.figure()
NameError: name 'plt' is not defined

----------------------------------------------------------------------
Ran 2 tests in 123.785s

FAILED (errors=1)


tensor([[[[0.6539, 0.6871, 0.6093,  ..., 0.6188, 0.6931, 0.6763],
          [0.6470, 0.5947, 0.5041,  ..., 0.4452, 0.5036, 0.6048],
          [0.6251, 0.6854, 0.5554,  ..., 0.5942, 0.6410, 0.5817],
          ...,
          [0.6923, 0.6742, 0.6931,  ..., 0.6849, 0.6289, 0.5847],
          [0.5452, 0.6760, 0.6906,  ..., 0.5764, 0.4018, 0.3620],
          [0.5624, 0.6746, 0.6780,  ..., 0.6094, 0.6683, 0.5042]]],


        [[[0.6498, 0.6918, 0.6255,  ..., 0.5922, 0.6930, 0.6882],
          [0.6851, 0.6618, 0.6026,  ..., 0.4693, 0.4991, 0.5838],
          [0.6745, 0.6906, 0.6815,  ..., 0.5886, 0.6730, 0.5786],
          ...,
          [0.6911, 0.6521, 0.6816,  ..., 0.6091, 0.5632, 0.5756],
          [0.5697, 0.6675, 0.6926,  ..., 0.6106, 0.5296, 0.4049],
          [0.6418, 0.6809, 0.6630,  ..., 0.5275, 0.6912, 0.4833]]],


        [[[0.6119, 0.6922, 0.6509,  ..., 0.6734, 0.6901, 0.6841],
          [0.6548, 0.5971, 0.5724,  ..., 0.5675, 0.4399, 0.6045],
          [0.6897, 0.6879, 0.6780,  ..